<a href="https://colab.research.google.com/github/YdoUcare-qm/MaskedBERT/blob/main/notebooks/TrainingWorkFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 18.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 18.0.0 which is incompatible.


In [ ]:
import datasets
from datasets import load_dataset

In [ ]:
ds=load_dataset("Yelp/yelp_review_full")
ds

README.md:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/299M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

## Tokenizer

In [ ]:
import transformers
from transformers import AutoTokenizer

In [ ]:
bert_tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")
bert_tokenizer

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

## Model

In [ ]:
from transformers import BertConfig,BertForMaskedLM

In [ ]:
config=BertConfig(vocab_size=30522, hidden_size=768, num_hidden_layers=12,
                  num_attention_heads=12, intermediate_size=3072, hidden_act='gelu',
                  hidden_dropout_prob=0.1, attention_probs_dropout_prob=0.1, max_position_embeddings=512,
                  type_vocab_size=2, initializer_range=0.02, layer_norm_eps=1e-12, pad_token_id=0,
                  position_embedding_type='absolute', use_cache=True, classifier_dropout=None)

In [ ]:
model=BertForMaskedLM(config)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [ ]:
from datasets import concatenate_datasets

### Chunking the Dataset into samples of 512

In [ ]:
concated=concatenate_datasets([ds['train'],ds['test']])
concated

Dataset({
    features: ['label', 'text'],
    num_rows: 700000
})

In [ ]:
def concatenate_and_chunk(examples):
    examples = bert_tokenizer(examples['text'])
    concat_ip_ids = sum([sequence for sequence in examples['input_ids']],[])
    total_tokens = len(concat_ip_ids)
    num_ctx_len_samples = total_tokens//512
    list_ip_ids = []
    list_att_mask = []
    for i in range(0,num_ctx_len_samples):
        input_ids = concat_ip_ids[i*512:(i+1)*512]
        att_mask = [1 for _ in range(512)]
        list_ip_ids.append(input_ids)
        list_att_mask.append(att_mask)
    return {'input_ids':list_ip_ids,'attention_mask':list_att_mask}

In [ ]:
ds_chunked=concated.map(concatenate_and_chunk,batched=True,batch_size=1000,remove_columns=['text','label'],num_proc=2)
ds_chunked

Map (num_proc=2):   0%|          | 0/700000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (539 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1019 > 512). Running this sequence through the model will result in indexing errors


Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 246695
})

In [ ]:
bert_tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")
bert_tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
ds_new=ds_chunked.train_test_split(test_size=0.05,seed=42)
ds_new

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 234360
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 12335
    })
})

In [ ]:
from transformers import DataCollatorForLanguageModeling

In [ ]:
collator=DataCollatorForLanguageModeling(tokenizer=bert_tokenizer,
                                         mlm=True,mlm_probability=0.2)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
trainloader=DataLoader(ds_new['train'],batch_size=16,collate_fn=collator)
testloader=DataLoader(ds_new['test'],batch_size=16,collate_fn=collator)

In [ ]:
config=BertConfig(num_hidden_layers=6,intermediate_size=1536,hidden_size=384)
model=BertForMaskedLM(config)

In [ ]:
trainloader=DataLoader(ds_new['train'],batch_size=8,collate_fn=collator)
testloader=DataLoader(ds_new['test'],batch_size=8,collate_fn=collator)

In [ ]:
from transformers import Trainer,TrainingArguments

In [ ]:
args=TrainingArguments(
    output_dir='/content/sample_data',
    per_device_train_batch_size=8,
    num_train_epochs=1,
    save_steps=1000,
    save_total_limit=2,
    report_to="none"
)

In [ ]:
trainer=Trainer(
    model=model,
    args=args,
    train_dataset=ds_new['train'],
    data_collator=collator
)

In [ ]:
trainer.train()

Step,Training Loss
500,7.010600
1000,6.221900
1500,6.125200
2000,6.062200
2500,6.032400
3000,5.997700
3500,5.974800
4000,5.945600
4500,5.932400
5000,5.923600


TrainOutput(global_step=29295, training_loss=5.863220533614151, metrics={'train_runtime': 6988.2403, 'train_samples_per_second': 33.536, 'train_steps_per_second': 4.192, 'total_flos': 7794712146493440.0, 'train_loss': 5.863220533614151, 'epoch': 1.0})

In [ ]:
import torch

In [ ]:
test="i was on that [MASK] "
input=bert_tokenizer(test,return_tensors='pt')
op=model(**input)
token_logits=op.logits
mask_token_index=torch.where(input['input_ids']==bert_tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens=torch.topk(mask_token_logits,5,dim=1).indices[0].tolist()
for token in top_5_tokens:
  print(test.replace("[MASK]",bert_tokenizer.decode([token])))

i was on that i 
i was on that was 
i was on that the 
i was on that . 
i was on that and 
